In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import GRU, Dense, Activation
from sklearn.metrics import mean_squared_error

In [2]:
train = pd.read_csv("../../Data/Kaggle/StoreSales/processed_train_v2.csv")
test = pd.read_csv("../../Data/Kaggle/StoreSales/processed_test_v2.csv")

In [3]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

In [4]:
train.set_index('date', inplace=True)
test.set_index('date', inplace=True)

In [5]:
train.head()

,id,store_nbr,family,sales,onpromotion,typeholiday,dcoilwtico,city,state,typestores,cluster,day_of_week,day,month,year
date,,,,,,,,,,,,,,,
2013-01-01,0,1,0,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
2013-01-01,1,1,1,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
2013-01-01,2,1,2,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
2013-01-01,3,1,3,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
2013-01-01,4,1,4,0.0,0,3,93.14,18,12,3,13,2,1,1,2013


In [6]:
test.head()

,id,store_nbr,family,onpromotion,typeholiday,dcoilwtico,city,state,typestores,cluster,day_of_week,day,month,year
date,,,,,,,,,,,,,,
2017-08-16,3000888,1,0,0,4,46.8,18,12,3,13,3,16,8,2017
2017-08-16,3000889,1,1,0,4,46.8,18,12,3,13,3,16,8,2017
2017-08-16,3000890,1,2,2,4,46.8,18,12,3,13,3,16,8,2017
2017-08-16,3000891,1,3,20,4,46.8,18,12,3,13,3,16,8,2017
2017-08-16,3000892,1,4,0,4,46.8,18,12,3,13,3,16,8,2017


In [7]:
X = train.drop(['id','sales'], axis=1)
y = train['sales']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
time_steps = 10

In [11]:
X_train_reshaped = []
y_train_reshaped = []

In [12]:
for i in range(len(X_train_scaled) - time_steps):
    X_train_reshaped.append(X_train_scaled[i:i + time_steps])
    y_train_reshaped.append(y_train.values[i + time_steps])

In [13]:
X_train_reshaped = np.array(X_train_reshaped)
y_train_reshaped = np.array(y_train_reshaped)

In [14]:
X_test_reshaped = []
y_test_reshaped = []

In [15]:
for i in range(len(X_test_scaled) - time_steps):
    X_test_reshaped.append(X_test_scaled[i:i + time_steps])
    y_test_reshaped.append(y_test.values[i + time_steps])

In [16]:
X_test_reshaped = np.array(X_test_reshaped)
y_test_reshaped = np.array(y_test_reshaped)

In [ ]:
optimizer='adam'


In [17]:
model = Sequential()
model.add(GRU(100, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dense(units=1, activation='linear'))
model.compile(optimizer=optimizer, loss='mean_squared_error')

In [18]:
model.fit(X_train_reshaped, y_train_reshaped, epochs=10, batch_size=10000, validation_split=0.2)

Epoch 1/10
193/193 [==============================] - 35s 176ms/step - loss: 1317122.3750 - val_loss: 1322025.0000
Epoch 2/10
193/193 [==============================] - 36s 189ms/step - loss: 1302334.1250 - val_loss: 1309857.3750
Epoch 3/10
193/193 [==============================] - 32s 167ms/step - loss: 1290810.2500 - val_loss: 1298830.7500
Epoch 4/10
193/193 [==============================] - 33s 169ms/step - loss: 1280228.5000 - val_loss: 1288849.6250
Epoch 5/10
193/193 [==============================] - 34s 175ms/step - loss: 1270761.3750 - val_loss: 1279879.3750
Epoch 6/10
193/193 [==============================] - 33s 169ms/step - loss: 1262223.0000 - val_loss: 1271796.5000
Epoch 7/10
193/193 [==============================] - 32s 166ms/step - loss: 1254528.2500 - val_loss: 1264512.0000
Epoch 8/10
193/193 [==============================] - 32s 166ms/step - loss: 1247601.1250 - val_loss: 1257963.8750
Epoch 9/10
193/193 [==============================] - 34s 178ms/step - loss: 124

In [19]:
test_loss = model.evaluate(X_test_reshaped, y_test_reshaped)
print(f'Test Loss: {test_loss}')

18756/18756 [==============================] - 19s 997us/step - loss: 1272325.1250
Test Loss: 1272325.125


In [20]:
y_pred = model.predict(X_test_reshaped)

18756/18756 [==============================] - 18s 927us/step


In [21]:
model.save('M18.h5')

/Users/yrrebeere/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
rmse = np.sqrt(mean_squared_error(y_test_reshaped, y_pred))
print("RMSE:", rmse)

RMSE: 1127.9735216183988
